In [33]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
from nltk.stem import PorterStemmer,WordNetLemmatizer

from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt


In [2]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
sample_submission=pd.read_csv('sample_submission.csv')

### EDA

In [3]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
sample_submission.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [6]:
print('Train dataset Info',train.info(),'\n\n')
print('Test dataset Info',test.info(),'\n\n')
print('sample_submission dataset Info',sample_submission.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB
Train dataset Info None 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB
Test dataset Info None 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dt

In [7]:
train['target'].value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

### preprocessing

In [8]:
x= train['text'].values
y= train['target'].values

In [9]:
print(len(x))
x

7613


array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [10]:
text='For detailed information on text preprocessing methods in NLP, visit https://www.nltk.org/.'
def preprocessing(sentence):
    
       
    ###   getting rid of URLs
    url_pattern = re.compile(r'http\S+|www\S+|https\S+')
    text = re.sub(url_pattern, '', sentence)
    
    ###   keep only letters
    text = re.sub('[^a-zA-Z]',' ',text)

    ###  turn all into lower case
    text = text.lower()
    
    ### tokenize
    text = nltk.word_tokenize(text)
    
    ###  remove stop words and stemmer
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    text = [ps.stem(word) for word in text if not word in all_stopwords]

    ### lemmatizer
    lemma = WordNetLemmatizer()
    
    text = [lemma.lemmatize(word, wordnet.VERB) for word in text ]
    text = [lemma.lemmatize(word, wordnet.ADJ) for word in text ]

    
    ### join words
    text = ' '.join(text)
    
    return text.strip()

In [11]:
cleaned_text = preprocessing(str(x[0]))
cleaned_text

'deed reason earthquak may allah forgiv us'

In [12]:
train['text'] = train['text'].astype(str)
train['text'] = train['text'].apply(preprocessing)

In [13]:
train.head()


,id,keyword,location,text,target
0,1,NaN,NaN,deed reason earthquak may allah forgiv us,1
1,4,NaN,NaN,forest fire near la rong sask canada,1
2,5,NaN,NaN,resid ask shelter place notifi offic evacu she...,1
3,6,NaN,NaN,peopl receiv wildfir evacu order california,1
4,7,NaN,NaN,get send photo rubi alaska smoke wildfir pour ...,1


In [14]:
train.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [16]:
x = train['text']

In [17]:
tf = TfidfVectorizer()
#### should pass the column
def vectorize(x):
    matrix = tf.fit_transform(x.tolist())
    return pd.DataFrame(matrix.toarray(),columns = tf.get_feature_names_out())

In [18]:
vectorized =vectorize(train['text'])

In [19]:
vectorized.head()

,aa,aaaa,aaaaaaallll,aaaaaand,aaarrrgghhh,aac,aal,aamir,aan,aannnnd,...,zoom,zotar,zouma,zourryart,zrnf,zss,zumiez,zurich,zxatheti,zzzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
##vectorized['y'] = train['target']

### training

In [21]:
from sklearn.model_selection import train_test_split
x= vectorized
y= train['target']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [22]:
def supervised_model(x_train,y_train,x_test,y_test,model):
    model.fit(x_train, y_train)
    y_pred= model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    return y_pred, accuracy
    

#### naive bayes

In [23]:
nb = GaussianNB()
y_gaussian_pred , gaussian_accuracy = supervised_model(x_train,y_train,x_test,y_test,nb)
print(gaussian_accuracy)

0.6007879185817465


#### logestic regression

In [24]:
lr = LogisticRegression()
y_logestic_reg_pred , logestic_reg_accuracy = supervised_model(x_train,y_train,x_test,y_test,lr)

print(logestic_reg_accuracy)

0.7905449770190414


#### support vector machine

In [25]:
svm =SVC()
y_svm_pred , svm_accuracy = supervised_model(x_train,y_train,x_test,y_test,svm)
print(svm_accuracy)

0.7977675640183848


#### neural network

In [26]:
clf = MLPClassifier()
y_NT_pred , NT_accuracy = supervised_model(x_train,y_train,x_test,y_test,clf)
print(NT_accuracy)

0.7301378857518056


### model evaluation


In [27]:
def model_evaluation(y_test,y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    cm = confusion_matrix(y_test, y_pred)
    return accuracy, precision, recall, f1, cm
    
    

#### model evaluation for logestic regression

In [40]:
model_evaluation(y_test,y_logestic_reg_pred)

(0.7905449770190414,
 0.7955425218262054,
 0.7905449770190414,
 0.7854983509963227,
 array([[786,  88],
        [231, 418]], dtype=int64))

#### model evaluation for support vector machine

In [35]:
model_evaluation(y_test, y_svm_pred)

(0.7977675640183848,
 0.8065838409974696,
 0.7977675640183848,
 0.7917093125212229,
 array([[803,  71],
        [237, 412]], dtype=int64))

#### model evaluation for naive bayes

In [37]:
model_evaluation(y_test, y_gaussian_pred)

(0.6007879185817465,
 0.6512469727909099,
 0.6007879185817465,
 0.5941779954698109,
 array([[401, 473],
        [135, 514]], dtype=int64))

#### model evaluation for neural network

In [39]:
model_evaluation(y_test, y_NT_pred)

(0.7301378857518056,
 0.7289110090579581,
 0.7301378857518056,
 0.7292563811513657,
 array([[683, 191],
        [220, 429]], dtype=int64))